# The `United States Counties` case study

During the 2020 US presidential election, the world and America alike were reminded that where an individual lives can best predict what they will decide for their future (that is, how they vote).

In [ ]:
#| output: false
#| echo: false

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

We will use the following data sources to create a dataset that allows us to perform voting analysis:

1. The US Department of Agriculture Economic Research Service (USDA ERS) [https://www.ers.usda.gov/data-products/county-level-data-sets/](https://www.ers.usda.gov/data-products/county-level-data-sets/) produces four datasets

    1. `Education.xls`,
    1. `PopulationEstimates.xls`,
    1. `PovertyEstimates.xls`, and
    1. `Unemployment.xlsx`


1. The US election results from Massachusetts Institute of Technology (MIT) election data [https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/VOQCHQ](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/VOQCHQ)

We will eventually integrate all this data into a single table.

However, how do we do so?

# Data integration

![](./img/datapreprocessing/integration.svg)

# Election data

In [ ]:
election_df = pd.read_csv('./datasets/census/countypres_2000-2020.csv')
election_df[:4]

In [ ]:
election_df.columns

# Profiling

In [ ]:
election_df.info()

# `mode`s of voting

- `mode` refers to the different ways that individuals had been able to participate in the election.

In [ ]:
plt.xticks(rotation='vertical')
election_df["mode"].hist()

# 

The `mode` `'TOTAL'` is the sum of all the other modes

1. Data selection: we can drop all the other rows that have modes other than `'TOTAL'`
1. Feature selection: since `mode` now has a single value, we can drop the column
1. Data cleaning: lowering the `county_name`

In [ ]:
election_df = election_df[election_df['mode'] == 'TOTAL']
election_df = election_df.drop(columns=['mode'])
election_df["county_name"] = election_df["county_name"].apply(lambda x: x.lower())
election_df[:4]

# `county_fips`

- County FIPS Codes are unique 5-digit codes that represent specific US counties
- We can drop them and keep only the county names

In [ ]:
election_df["county_fips"].hist()
election_df = election_df.drop(columns=['county_fips'])

# Checking the granularity of the analysis

How can we find the subject (primary key) of each row?

In [ ]:
election_df.groupby(['state', 'state_po']).count()[:8]

# Granularity: `State` + `County`

In [ ]:
election_df.groupby(['state', 'state_po', 'county_name']).count()[:8]

# Granularity: `State` + `County` + `Year`

In [ ]:
election_df.groupby(['state', 'state_po', 'county_name', 'year']).count()[:8]

# Granularity: `State` + `County` + `Year` + `Party`

In [ ]:
election_df.groupby(['state', 'state_po', 'county_name', 'year', 'party']).count()[:8]

# Feature engineering and aggregation: the `partisanism` attribute

- This is too detailed for our analysis
- We can aggregate `party` in the same year by constructing the new attribute `partisanism` = $\frac{republicans - democrats}{all~votes}$
    - partisan (adj): feeling or showing adherence to a particular party, faction, cause, or person
- We coarsen the granularity from (`State` + `County` + `Year` + `Party`) to (`State` + `County` + `Year`)

In [ ]:
# Group the election_df by state, county, and year
grouped_df = election_df.groupby(['state_po', 'county_name', 'year'])
# Function to calculate partisanism within each group
def calculate_partisanism(group):
    # Calculate votes for each party
    democrat_votes = group[group['party'] == 'DEMOCRAT']['candidatevotes'].sum()
    republican_votes = group[group['party'] == 'REPUBLICAN']['candidatevotes'].sum()
    # Calculate partisanism: Republicans as positive, Democrats as negative
    return (republican_votes - democrat_votes) / group["candidatevotes"].sum()
# Apply the function to each group
partisanism_df = grouped_df.apply(calculate_partisanism).reset_index(name='partisanism')
partisanism_df[partisanism_df["state_po"] == "NY"][:4]

# The `partisanism` time series

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 3))  # Adjust figure size as needed
i = 0
partisanism_df = partisanism_df.sort_values(by=["year", "county_name"])  # sort data by year
filtered_df = partisanism_df[partisanism_df["state_po"] == "NY"]  # consider only counties in NY
for county in filtered_df['county_name'].unique():  # Plot partisanism over time for each county
    county_data = filtered_df[filtered_df['county_name'] == county]
    plt.plot(county_data['year'], county_data['partisanism'], label=county, marker='o')
    if i == 9: break  # only plot the first 10 counties in NY
    i += 1
# Add labels and title
ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Partisanism', fontsize=12)
plt.xticks(ticks=filtered_df["year"].unique(), rotation=45)  # Rotate x-axis labels for better readability
ax.legend(title='County', bbox_to_anchor=(1.05, 1), loc='upper left')  # Move legend outside the plot
fig.tight_layout()

# Aggregating `partisanism`

- We have the partisanism value of presidential elections 2000, 2004, 2008, 2012, 2016, and 2020 for every county
- We can interpolate the data and obtain the mean and slope of partisanism across elections over 20 years
- We coarsen the granularity from (`State` + `County` + `Year`) to (`State` + `County`)

In [ ]:
#| echo: false

from scipy.optimize import curve_fit

# Define a linear function for curve fitting
def linearFunction(x, a, b): return a * x + b

# Function to compute mean and slope for each group (state_po, county_name)
def compute_mean_slope(wdf):
    n = len(wdf)
    if n <= 1:  # If there's only one data point or none, assign default values
        mean_partisanism = wdf['partisanism'].mean() if n == 1 else np.nan
        slope, intercept = np.nan, np.nan  # No linear interpolation with one point
    else:
        x_data = np.arange(n)  # x_data is range of available data points
        y_data = wdf['partisanism'].values  # y_data is the partisanism values
        try:
            p, _ = curve_fit(linearFunction, x_data, y_data)  # Curve fitting to find slope and intercept
            slope, intercept = p[0], p[1]  # Slope of the linear fit
            mean_partisanism = y_data.mean()
        except Exception as e:
            # print(f"Error in curve fitting for {wdf['state_po'].iloc[0]} {wdf['county_name'].iloc[0]}: {e}")
            slope, intercept, mean_partisanism = np.nan, np.nan, np.nan
    # Return a series with mean and slope
    return pd.Series({'mean': mean_partisanism, 'slope': slope, 'intercept': intercept})

# Group by state_po and county_name and apply the function to each group
partisanism_df = partisanism_df.groupby(['state_po', 'county_name']).apply(compute_mean_slope).reset_index()
partisanism_df.columns = ['State','County_Name', 'Mean_Partisanism', 'Slope_Partisanism', 'Intercept_Partisanism']
partisan_df = partisanism_df.set_index(['State', 'County_Name'])
partisan_df

# Visualizing the intercept

In [ ]:
#| echo: false

x_vals = np.arange(1, 7)  # Define a range of x-values to plot the lines (adjust as needed)
cur_df = partisan_df.reset_index()
i = 0
for idx, row in cur_df[cur_df["State"] == "NY"].iterrows():  # Plot a line for each group using the slope and intercept
    y_vals = row['Slope_Partisanism'] * x_vals + row['Intercept_Partisanism']
    ax.plot(np.arange(2000, 2024, 4), y_vals, label=f'{row["County_Name"]}', ls="--")  # Use state and county as labels
    if i == 9: break  # only plot the first 10 counties in NY
    i += 1

fig

# Education data

In [ ]:
edu_df = pd.read_excel('./datasets/census/Education.xls')
edu_df[:4]

# Do not consider the first 3 rows

In [ ]:
edu_df = pd.read_excel('./datasets/census/Education.xls', skiprows=4)
edu_df[:4]

#

In [ ]:
edu_df.info()

# Selecting some features

In [ ]:
edu_df = edu_df[['State', 'Area name', "Percent of adults with a bachelor's degree or higher, 2015-19"]]
edu_df[:4]

In [ ]:
edu_df = edu_df.rename({'Area name': 'Area_Name', "Percent of adults with a bachelor's degree or higher, 2015-19": "HigherEdPercent"}, axis=1)
edu_df = edu_df[edu_df["Area_Name"].apply(lambda x: x.endswith("County"))]
edu_df[:4]

# Poverty data

In [ ]:
pov_df = pd.read_excel('./datasets/census/PovertyEstimates.xls', skiprows=4)
print(pov_df.columns)
# 'POV04_2019' = Estimated percent of people of all ages in poverty 2019
# 'MEDHHINC_2019' = Estimate of median household income 2019
pov_df = pov_df[['Stabr', 'Area_name', 'PCTPOVALL_2019', 'MEDHHINC_2019']]
pov_df = pov_df.rename({'Area_name': 'Area_Name', 'Stabr': 'State'}, axis=1)
pov_df[:4]

# Unemployment data

In [ ]:
employment_df = pd.read_excel('./datasets/census/Unemployment.xlsx', skiprows=4)
employment_df = employment_df[['State', 'Area_name', 'Unemployment_rate_2019']]
employment_df[:4]

In [ ]:
employment_df['Area_Name'] = employment_df.Area_name.apply(lambda v: v[:v.find(',')] if ',' in v else v)
employment_df = employment_df.drop(columns = ["Area_name"], axis=1)
employment_df[:4]

# Population data

In [ ]:
pop_df = pd.read_excel('./datasets/census/PopulationEstimates.xls', skiprows=2)
print(pop_df.columns)
pop_df[:4]

#

In [ ]:
pop_df = pop_df[['State', 'Area_Name', 'CENSUS_2010_POP']]
pop_df = pop_df.rename({'CENSUS_2010_POP': "population"}, axis=1)
pop_df[:4]

# Integration

Now `edu_df`, `pop_df`, `pov_df`, `employment_df` all have the `State` and `Area_Name` columns

In [ ]:
print(f"edu_df: {edu_df.columns}")
print(f"pop_df: {pop_df.columns}")
print(f"pov_df: {pov_df.columns}")
print(f"employment_df: {employment_df.columns}")

Still, they require some manipulation.

- For instance, they contains data from both the US (aggregated data) and single counties (detailed data)

In [ ]:
for cur_df in [edu_df, pop_df, pov_df, employment_df]:
    cur_df.drop(cur_df.index[cur_df['Area_Name'].apply(lambda x: "County" not in x)], inplace=True)  # Only keep the counties
    cur_df['County_Name'] = cur_df['Area_Name'].apply(lambda x: x.replace(" County", "").lower())  # Lower the name of all counties
    cur_df.drop(columns = ['Area_Name'], inplace=True)  # Drop the column `Area_Name`
    cur_df.set_index(['State', 'County_Name'], inplace=True)  # Set the index of the table

#

In [ ]:
edu_df[:2]

In [ ]:
pop_df[:2]

In [ ]:
pov_df[:2]

In [ ]:
employment_df[:2]

# Also, we have `partisan_df`

In [ ]:
partisan_df

# Joining the sources

In [ ]:
county_df = pop_df.join(edu_df).join(pov_df).join(employment_df).join(partisan_df)
county_df[:4]

#

In [ ]:
county_df.info()

# Handling the missing values

In [ ]:
county_df.dropna(inplace=True)
county_df.info()

# Outliers

In [ ]:
for att in county_df.columns[:2]:
    fig, axs = plt.subplots(1, 2, sharey=False, figsize=(6, 2))
    county_df[att].plot.hist(ax=axs[0])
    county_df[att].plot.box(vert=True, ax=axs[1])
    axs[1].set_ylabel('Value')
    fig.tight_layout()

# Setting the `class` attribute

In [ ]:
def set_class(x):
    if x > 0.1: return "republican"
    elif x > -0.1: return "other"
    else: return "democratic"

county_df["class"] = county_df["Mean_Partisanism"].apply(set_class)
county_df[:4]

In [ ]:
#| echo: false
#| output: false

colors = {"republican": "blue", "democratic": "red", "other": "grey"}
def color(s):
    return colors[s]

# Distribution of `class`

In [ ]:
county_df["class"].hist()

# Checking for data correlations

In [ ]:
county_df_no_p = county_df[[x for x in county_df.columns if "Partisanism" not in x]]
sns.pairplot(county_df_no_p, diag_kind="hist", hue="class", markers=".", height=1.5, palette=colors)

In [ ]:
#| echo: false
#| output: false

plt.figure(figsize=(10,10))
sns.heatmap(county_df.drop(columns=["class"]).corr(), linewidths=.5, annot=True, cmap='viridis')
plt.tight_layout()

# Using PCA to visualize the dataset

In [ ]:
from sklearn.decomposition import PCA

Xs = county_df.drop(columns=["Mean_Partisanism", "Slope_Partisanism", "Intercept_Partisanism", "class"])
Xs = (Xs -Xs.mean()) / Xs.std()
pca = PCA(n_components=2)
pca.fit(Xs)
Xs_t = pd.DataFrame(pca.transform(Xs), index = Xs.index)
Xs_t.columns = ['PC{}'.format(i) for i in range(1, len(Xs_t.columns) + 1)]
Xs_t[:4]

In [ ]:
#| echo: false
#| output: false

total_variance = Xs_t.var().sum()
explanation_df = pd.DataFrame({'variance_percentage': Xs_t.var() / total_variance, 'cumulative_variance_percentage': Xs_t.var().cumsum() / total_variance})
explanation_df

# Visualizing the data

In [ ]:
Xs_t.plot.scatter(x='PC1', y='PC2', c=county_df["class"].apply(color), sharex=False, colormap ='gray', marker='.',figsize=(10, 10))

for i, row in Xs_t.iterrows():
    if row.PC1 > 6.5 or row.PC1 < -4.5:
        plt.annotate(i, (row.PC1, row.PC2), size=8)
    elif row.PC2 < -7 or row.PC2 > 3.5:
        plt.annotate(i, (row.PC1 - 1, row.PC2), size=8)

# Doing machine learning

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def dtree(county_df, max_depth, print_text=False, plot=True):
    X = county_df.drop(columns='class')
    y = county_df['class']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)  # Split the data into training and testing sets
    clf = DecisionTreeClassifier(max_depth=max_depth)  
    clf.fit(X_train, y_train)  # Train the decision tree on the training data
    y_pred = clf.predict(X_test)  # Make predictions on the test data
    if plot: plot_tree(clf, feature_names=X.columns, class_names=clf.classes_, filled=True, rounded=True)
    if print_text: tree_text = print(export_text(clf, feature_names=list(X.columns)))
    return accuracy_score(y_test, y_pred)  # Compute the accuracy

dtree(county_df, max_depth=3)

# Another decision tree

In [ ]:
dtree(county_df.drop(columns=["Mean_Partisanism", "Slope_Partisanism", "Intercept_Partisanism"]), max_depth=3, print_text=True, plot=False)

In [ ]:
# Can you do better?